In [1]:
import numpy as np
import pandas as pd
import math
import os
from utils import *
from sklearn.model_selection import train_test_split
from models import SVDplusplus


df = pd.read_csv("data/data_train.csv")
args = read_config('config_models.yaml')


model = SVDplusplus(args)

train, test = train_test_split(df, test_size=args.test_set_size, random_state=args.random_state)

model.train(train)

model.predict(test)


 processing epoch 0
 processing epoch 1
 processing epoch 2
 processing epoch 3
 processing epoch 4
 processing epoch 5
 processing epoch 6
 processing epoch 7
 processing epoch 8
 processing epoch 9
 processing epoch 10
 processing epoch 11
 processing epoch 12
 processing epoch 13
 processing epoch 14
 processing epoch 15
 processing epoch 16
 processing epoch 17
 processing epoch 18
 processing epoch 19
